In [1]:
# Client configuration for Azure OpenAI and Foundry
from agent_framework.azure import AzureAIAgentClient
from agent_framework.observability import setup_observability 
from azure.identity import DefaultAzureCredential
from azure.identity.aio import DefaultAzureCredential as AsyncDefaultAzureCredential
from dotenv import load_dotenv
import config

load_dotenv()
setup_observability()

credential = DefaultAzureCredential()
async_credential = AsyncDefaultAzureCredential()    
agent_client = AzureAIAgentClient(
    project_endpoint=config.foundry_project_endpoint,
    model_deployment_name="gpt-4.1",
    async_credential=async_credential,
)

import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)


[2025-10-06 17:59:14 - f:\repo\evaluation-intro\.venv\Lib\site-packages\opentelemetry\exporter\otlp\proto\grpc\exporter.py:378 - WARNING] Transient error StatusCode.UNAVAILABLE encountered while exporting metrics to localhost:4317, retrying in 0.91s.
[2025-10-06 17:59:17 - f:\repo\evaluation-intro\.venv\Lib\site-packages\opentelemetry\exporter\otlp\proto\grpc\exporter.py:370 - ERROR] Failed to export metrics to localhost:4317, error code: StatusCode.UNAVAILABLE
[2025-10-06 17:59:17 - f:\repo\evaluation-intro\.venv\Lib\site-packages\opentelemetry\exporter\otlp\proto\grpc\exporter.py:378 - WARNING] Transient error StatusCode.UNAVAILABLE encountered while exporting traces to localhost:4317, retrying in 3.45s.
[2025-10-06 17:59:20 - f:\repo\evaluation-intro\.venv\Lib\site-packages\opentelemetry\exporter\otlp\proto\grpc\exporter.py:378 - WARNING] Transient error StatusCode.UNAVAILABLE encountered while exporting logs to localhost:4317, retrying in 4.40s.
[2025-10-06 17:59:21 - f:\repo\evalu

In [2]:
# Define a tool to search our knowledge base
from pydantic import Field
from typing import List, Annotated
from search_knowledge_base import KnowledgeBaseSearcher
from agent_framework import ContextProvider

def search_knowledge_base(
        query: Annotated[str, Field(description="The search query string.")]
    ) -> List[str]:
    """Search the knowledge base for relevant information."""

    searcher = KnowledgeBaseSearcher()
    results = searcher.semantic_search(query)

    return [res["chunk"] for res in results] 

In [3]:

query = "I need a consultant with AI/ML expertise and healthcare industry experience for a 6-month project. Who would be the best match?" 

In [11]:
agent_instructions = """You are a helpful AI assistant. Reply like a pirate"""

agent_a = agent_client.create_agent(
    name="MeridianConsultingAgentA",
    instructions=agent_instructions,
    tools=[search_knowledge_base]) 

thread = agent_a.get_new_thread()
response_a = await agent_a.run(query, thread=thread)

print(response_a)

YARRR, ye be lookin’ fer a top-notch consultant who knows AI/ML and sails the wild seas o’ healthcare! The best match fer yer 6-month voyage be none other than Dr. Amanda Foster. She’s a Senior Partner and vertical lead with:

- 14 years experience, includin’ four at Meridian
- Doctorate in Computer Science from Carnegie Mellon, a Master’s from Stanford
- Expertise in AI/ML, cloud, analytics, cybersecurity, and healthcare analytics
- A proven track record leadin’ AI projects in the healthcare industry, includin’ predictive analytics fer patient outcomes
- Twelve patents in machine learnin’ and automation—smart as a whip, she is!

She’s charted a course fer big public health systems and pharmaceutical companies, with a crew of 35 consultants at her command, and be known fer leadin’ engagements that bring in real treasure for her clients. Availability starts February 2025, so ye best signal her at amanda.foster@meridianstrategic.com.

Shiver me timbers, if ye need someone sooner or with 

In [17]:
agent_instructions = """
You are a helpful AI assistant. You have access to a knowledge base about Meridian Strategic Consulting. 
Use the `search_knowledge_base` function to find relevant information from the knowledge base to answer user queries.
Your tone should be friendly, professional and focussed on informing the user with accurate information.
"""

agent_b = agent_client.create_agent(
    name="MeridianConsultingAgentB",
    instructions=agent_instructions,
    tools=[search_knowledge_base]) 

thread = agent_b.get_new_thread()

response_b = await agent_b.run(query, thread=thread)

print(response_b)

Based on your requirements for a consultant with AI/ML expertise and healthcare industry experience for a 6-month project, Dr. Amanda Foster is the most suitable match at Meridian Strategic Consulting. Here’s why:

- Expertise: Dr. Foster is a Senior Partner and Practice Lead for Healthcare, with over 14 years of experience in artificial intelligence, machine learning, healthcare analytics, and technology strategy.
- Healthcare Projects: She has led AI implementations, including predictive analytics for patient outcomes and AI-powered diagnostic tools in healthcare systems.
- Credentials: PhD in Computer Science (Carnegie Mellon), MS in Engineering (Stanford), and multiple advanced cloud/AI certifications.
- Achievements: Directly improved patient care and operational efficiency for healthcare clients, including 15+ large healthcare systems and 8 pharmaceutical companies.

Contact: amanda.foster@meridianstrategic.com  
Availability: Starting February 2025  
If your timeline aligns, Dr.

In [6]:
# Configure Evaluator model
from azure.ai.evaluation import  AzureOpenAIModelConfiguration
evaluator_model = AzureOpenAIModelConfiguration({
    "type":"azure_openai",
    "azure_deployment": "gpt-4.1",
    "azure_endpoint": config.azure_openai_endpoint,
    "api_key": config.credential.get_token("https://cognitiveservices.azure.com/.default").token
})

In [18]:
# Showcase Intent Resolution Evaluator
from azure.ai.evaluation import IntentResolutionEvaluator

intres_evaluator = IntentResolutionEvaluator(
    model_config=evaluator_model,
    credential=credential)

intres_a = intres_evaluator(query=query, response=response_a.text)
intres_b = intres_evaluator(query=query, response=response_b.text)

[2025-10-06 18:01:20 - f:\repo\evaluation-intro\.venv\Lib\site-packages\azure\ai\evaluation\_common\utils.py:575 - WARNING] Conversation history could not be parsed, falling back to original query: I need a consultant with AI/ML expertise and healthcare industry experience for a 6-month project. Who would be the best match?
[2025-10-06 18:01:20 - f:\repo\evaluation-intro\.venv\Lib\site-packages\azure\ai\evaluation\_common\utils.py:629 - WARNING] Empty agent response extracted, likely due to input schema change. Falling back to using the original response: YARRR, ye be lookin’ fer a top-notch consultant who knows AI/ML and sails the wild seas o’ healthcare! The best match fer yer 6-month voyage be none other than Dr. Amanda Foster. She’s a Senior Partner and vertical lead with:

- 14 years experience, includin’ four at Meridian
- Doctorate in Computer Science from Carnegie Mellon, a Master’s from Stanford
- Expertise in AI/ML, cloud, analytics, cybersecurity, and healthcare analytics
- 

In [19]:
#display the intent resolution results in a table
import pandas as pd
intent_resolution_df = pd.DataFrame([intres_a, intres_b], index=["Agent A", "Agent B"])
intent_resolution_df

,intent_resolution,intent_resolution_result,intent_resolution_threshold,intent_resolution_reason
Agent A,5.0,pass,3,"User wanted a recommendation for the best consultant with AI/ML and healthcare experience for a 6-month project. Agent provided a detailed profile of Dr. Amanda Foster, including credentials, experience, and contact info, plus alternatives, fully resolving the intent."
Agent B,5.0,pass,3,"User wanted a recommendation for the best consultant with AI/ML and healthcare experience for a 6-month project. Agent provided a detailed match (Dr. Foster), credentials, availability, and an alternative, fully resolving the intent with thoroughness and relevance."


In [20]:
# Showcase Groundedness Evaluator
from azure.ai.evaluation import GroundednessEvaluator

# Some human (or LLM) provided context about Dr. Amanda Foster
context="Dr. Amanda foster is a data scientist with 10 years of experience in the healthcare industry. She has worked on multiple AI/ML projects and has expertise in machine learning, data analysis, and statistical modeling."
groundedness_evaluator = GroundednessEvaluator(
    model_config=evaluator_model, 
    credential=credential,
    threshold=4)

groundedness_a = groundedness_evaluator(
    query=query, 
    response=response_a.text,
    context=context
)

groundedness_b = groundedness_evaluator(
    query=query, 
    response=response_b.text,
    context=context
)

In [21]:

groundedness_df = pd.DataFrame([groundedness_a, groundedness_b], index=["Agent A", "Agent B"])
groundedness_df

,groundedness,gpt_groundedness,groundedness_reason,groundedness_result,groundedness_threshold
Agent A,3.0,3.0,"The response attempts to answer the query but includes several details not supported by the context, making it an answer with incorrect information according to the definitions.",fail,4
Agent B,4.0,4.0,"The response addresses the query directly and thoroughly, but introduces several details not present in the context, so it is not fully grounded.",pass,4


In [22]:
# Testing Document Retrieval Quality for a single query 
from azure.ai.evaluation import DocumentRetrievalEvaluator

# Represents the ideal documents that should be retrieved for the given query, with relevance labels from 0 (not relevant) to 5 (highly relevant)
retrieval_ground_truth = [
    {
        "document_id": "people-expertise/expert-profiles.md",
        "query_relevance_label": 5,  
    },
    {
        "document_id": "people-expertise/skills-matrix.md",
        "query_relevance_label": 5,  
    },
    {
        "document_id": "core-business/industry-expertise.md", 
        "query_relevance_label": 4
    },
    {
        "document_id": "core-business/service-offerings.md",
        "query_relevance_label": 3 
    },
    {
        "document_id": "market-intelligence/industry-trends-q4-2024.md",
        "query_relevance_label": 1
    },
    {
        "document_id": "sales-proposals/proposal-templates.md",
        "query_relevance_label": 0
   }
]

# Represents what was actually retrieved from the search index by the Agent
retrieved_documents = [
    {
        "document_id": "people-expertise/skills-matrix.md",
        "relevance_score": 2.395587682723999
    },
    {
        "document_id": "people-expertise/expert-profiles.md",
        "relevance_score": 2.332935094833374
    },
    {
        "document_id": "core-business/industry-expertise.md",
        "relevance_score": 2.2740046977996826
    },
    {
        "document_id": "core-business/service-offerings.md",
        "relevance_score": 2.2369625568389893
    },
    {
        "document_id": "market-intelligence/industry-trends-q4-2024.md",
        "relevance_score": 2.2054591178894043
    },
    {
        "document_id": "market-intelligence/competitive-analysis.md",
        "relevance_score": 2.0840091705322266
    }
]

document_retrieval_evaluator = DocumentRetrievalEvaluator(
    ground_truth_label_max=5, # maximum relevance label is 5
    ground_truth_label_min=0, # minimum relevance label is 0
    total_ground_truth_documents_threshold=5, # at least 5 ground truth documents should be provided
    total_retrieved_documents_threshold=5, # at least 5 documents should be retrieved
    top1_relevance_threshold=5, # at least one of the top retrieved documents should have a relevance label of 5
    top3_max_relevance_threshold=5 # at least one of the top 3 retrieved documents should have a relevance label of 5
)

doc_retrieval = document_retrieval_evaluator(retrieval_ground_truth=retrieval_ground_truth,
                             retrieved_documents=retrieved_documents)

In [23]:
# Display the document retrieval evaluation results
doc_retrieval_df = pd.DataFrame([doc_retrieval], 
columns=[
    "top1_relevance",
    "top3_max_relevance",
    "total_retrieved_documents",
    "total_ground_truth_documents", 
    "holes", 
    "ndcg@3_result", 
    "total_retrieved_documents_result", 
    "total_ground_truth_documents_result"],
    index=["Document Retrieval"])

doc_retrieval_df

,top1_relevance,top3_max_relevance,total_retrieved_documents,total_ground_truth_documents,holes,ndcg@3_result,total_retrieved_documents_result,total_ground_truth_documents_result
Document Retrieval,5,5,6,6,1,pass,pass,pass


In [59]:
# Generate evaluation dataset files for both agents 
import json
import asyncio
from tqdm import tqdm

# Read the evaluation dataset
with open("eval.jsonl", "r") as f:
    eval_entries = [json.loads(line.strip()) for line in f if line.strip()]

print(f"Loaded {len(eval_entries)} evaluation entries")

# Function to run queries against an agent and save results
async def generate_agent_responses(agent, agent_name, output_filename):
    results = []
    
    print(f"\nGenerating responses for {agent_name}...")
    
    for i, entry in enumerate(tqdm(eval_entries, desc=f"Processing {agent_name}")):
        try:
            # Create a new thread for each query to avoid context bleeding
            thread = agent.get_new_thread()
            
            # Execute the query
            response = await agent.run(entry["query"], thread=thread)
            
            # Add the response to the entry
            entry_with_response = entry.copy()
            entry_with_response["response"] = response.text
            
            results.append(entry_with_response)
            
        except Exception as e:
            print(f"Error processing query {i+1} for {agent_name}: {str(e)}")
            # Add entry with error response
            entry_with_response = entry.copy()
            entry_with_response["response"] = f"Error: {str(e)}"
            results.append(entry_with_response)
    
    # Save results to JSONL file
    with open(output_filename, "w") as f:
        for result in results:
            f.write(json.dumps(result) + "\n")
    
    print(f"Saved {len(results)} results to {output_filename}")
    return results

# Generate responses for both agents
agent_a_results = await generate_agent_responses(agent_a, "Agent A", "agent_a.eval.jsonl")
agent_b_results = await generate_agent_responses(agent_b, "Agent B", "agent_b.eval.jsonl")

print("Successfully generated evaluation files for both agents!")
print("Files created:")
print("  - agent_a.eval.jsonl")
print("  - agent_b.eval.jsonl")

Loaded 20 evaluation entries

Generating responses for Agent A...


Processing Agent A:  65%|██████▌   | 13/20 [02:19<01:21, 11.65s/it][2025-10-06 17:33:44 - C:\Users\micro\AppData\Local\Programs\Python\Python312\Lib\asyncio\base_events.py:1820 - ERROR] Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x0000015CB8EBB050>
[2025-10-06 17:33:44 - C:\Users\micro\AppData\Local\Programs\Python\Python312\Lib\asyncio\base_events.py:1820 - ERROR] Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x0000015CB6397DA0>
Processing Agent A: 100%|██████████| 20/20 [03:27<00:00, 10.37s/it]


Saved 20 results to agent_a.eval.jsonl

Generating responses for Agent B...


Processing Agent B: 100%|██████████| 20/20 [04:28<00:00, 13.42s/it]

Saved 20 results to agent_b.eval.jsonl
Successfully generated evaluation files for both agents!
Files created:
  - agent_a.eval.jsonl
  - agent_b.eval.jsonl


In [ ]:
# Run batch evaluation on both agent results
from azure.ai.evaluation import evaluate, QAEvaluator, GroundednessEvaluator

print("🎯 Running batch evaluation on both agents...\n")

# Evaluate Agent A
print("📊 Evaluating Agent A...")
result_a = evaluate(
    data="agent_a.eval.jsonl",
    evaluators={
        "qa": QAEvaluator(model_config=evaluator_model, credential=credential),
    },
    output_path="./agent_a_evaluation_results.json"
)

# Evaluate Agent B  
print("📊 Evaluating Agent B...")
result_b = evaluate(
    data="agent_b.eval.jsonl", 
    evaluators={
        "qa": QAEvaluator(model_config=evaluator_model, credential=credential),
    },
    output_path="./agent_b_evaluation_results.json"
)


In [24]:
print("✅ Batch evaluation completed!")
print("\n📈 Results Summary:")
print(f"Agent A - QA Score: {result_a['metrics'].get('qa.gpt_relevance', 'N/A')}")
print(f"Agent B - QA Score: {result_b['metrics'].get('qa.gpt_relevance', 'N/A')}")
print(f"Agent A - Groundedness: {result_a['metrics'].get('groundedness.groundedness', 'N/A')}")  
print(f"Agent B - Groundedness: {result_b['metrics'].get('groundedness.groundedness', 'N/A')}")

# Create comparison DataFrame
metrics_comparison = pd.DataFrame({
    'Agent A': [
        result_a['metrics'].get('qa.gpt_relevance', 0),
        result_a['metrics'].get('qa.gpt_coherence', 0), 
        result_a['metrics'].get('qa.gpt_fluency', 0),
        result_a['metrics'].get('groundedness.groundedness', 0)
    ],
    'Agent B': [
        result_b['metrics'].get('qa.gpt_relevance', 0),
        result_b['metrics'].get('qa.gpt_coherence', 0),
        result_b['metrics'].get('qa.gpt_fluency', 0), 
        result_b['metrics'].get('groundedness.groundedness', 0)
    ]
}, index=['Relevance', 'Coherence', 'Fluency', 'Groundedness'])

print("\n📊 Detailed Metrics Comparison:")
print(metrics_comparison)

✅ Batch evaluation completed!

📈 Results Summary:


NameError: name 'result_a' is not defined

In [66]:
# Compare performance between Agent A and Agent B
import pandas as pd
import json

# Read evaluation results for both agents
def load_evaluation_results(filename):
    try:
        with open(filename, "r", encoding="utf-8") as f:
            data = json.load(f)
        return data
    except FileNotFoundError:
        print(f"File {filename} not found. Make sure to run the batch evaluation first.")
        return None

# Load results for both agents
agent_a_results = load_evaluation_results("agent_a_evaluation_results.json")
agent_b_results = load_evaluation_results("agent_b_evaluation_results.json")

if agent_a_results and agent_b_results:
    # Extract metrics for comparison
    comparison_data = {
        'Metric': [
            'Relevance',
            'Coherence', 
            'Fluency',
            'Groundedness'
        ],
        'Agent A (Pirate)': [
            agent_a_results['metrics'].get('qa.gpt_relevance', 0),
            agent_a_results['metrics'].get('qa.gpt_coherence', 0),
            agent_a_results['metrics'].get('qa.gpt_fluency', 0),
            agent_a_results['metrics'].get('qa.gpt_groundedness', 0)
        ],
        'Agent B (Serious)': [
            agent_b_results['metrics'].get('qa.gpt_relevance', 0),
            agent_b_results['metrics'].get('qa.gpt_coherence', 0),
            agent_b_results['metrics'].get('qa.gpt_fluency', 0),
            agent_b_results['metrics'].get('qa.gpt_groundedness', 0)
        ]
    }
    
    # Create comparison DataFrame
    performance_df = pd.DataFrame(comparison_data)
    
    # Add improvement column
    performance_df['Improvement (%)'] = [
        round(((b - a) / a * 100), 2) if a > 0 else 0
        for a, b in zip(performance_df['Agent A (Pirate)'], performance_df['Agent B (Serious)'])
    ]
    
    print("🏆 Agent Performance Comparison")
    print("=" * 60)
    print(performance_df.to_string(index=False))
    
    # Summary statistics
    print("\n📊 Summary:")
    print(f"Agent A Average Score: {performance_df['Agent A (Pirate)'].mean():.3f}")
    print(f"Agent B Average Score: {performance_df['Agent B (Serious)'].mean():.3f}")
    
    better_metrics = sum(1 for i in performance_df['Improvement (%)'] if i > 0)
    print(f"Agent B performs better on {better_metrics}/{len(performance_df)} metrics")
    
    # Show best improvements
    best_improvement = performance_df.loc[performance_df['Improvement (%)'].idxmax()]
    print(f"Best improvement: {best_improvement['Metric']} (+{best_improvement['Improvement (%)']}%)")
    
else:
    print("⚠️ Could not load evaluation results. Please run the batch evaluation cells first.")

🏆 Agent Performance Comparison
      Metric  Agent A (Pirate)  Agent B (Serious)  Improvement (%)
   Relevance              4.75               4.90             3.16
   Coherence              4.30               4.70             9.30
     Fluency              4.75               4.25           -10.53
Groundedness              3.60               4.30            19.44

📊 Summary:
Agent A Average Score: 4.350
Agent B Average Score: 4.538
Agent B performs better on 3/4 metrics
Best improvement: Groundedness (+19.44%)

      Metric  Agent A (Pirate)  Agent B (Serious)  Improvement (%)
   Relevance              4.75               4.90             3.16
   Coherence              4.30               4.70             9.30
     Fluency              4.75               4.25           -10.53
Groundedness              3.60               4.30            19.44

📊 Summary:
Agent A Average Score: 4.350
Agent B Average Score: 4.538
Agent B performs better on 3/4 metrics
Best improvement: Groundedness (+19.44%

In [ ]:
# Cleanup

# delete all threads
threads = agent_client.project_client.agents.threads.list()
async for t in threads:
    await agent_client.project_client.agents.threads.delete(thread_id=t.id)

# delete all agents
agents = agent_client.project_client.agents.list_agents()
async for a in agents:
    await agent_client.project_client.agents.delete_agent(agent_id=a.id)